# 1. <u>Project summary</u>

## What is your name?

Michaël Lambé

## What E-mail address do you use to sign in to Udacity?

mic0331@gmail.com

## What area of the world you used for your project? Post a link to the map position and write a short description. Note that the osm file of the map should be at least 50MB.

Map Area: Brussels, BE, Belgium
<img src="pictures/Brussels.png">

I chose the most known city of Belgium as this particular place because it is the capital of my country, I know it well and would like its map to be improved in quality!

https://s3.amazonaws.com/metro-extracts.mapzen.com/brussels_belgium.osm.bz2

## Is there a list  of  Web sites, books, forums, blog posts, github repositories etc that you referred to or used  in this  submission (Add N/A if you did not use  such resources)?

The following book has been used as a complementary reference for this project : <a href="http://www.amazon.com/Python-Data-Analysis-Wrangling-IPython/dp/1449319793/ref=sr_1_1?ie=UTF8&qid=1426059590&sr=8-1&keywords=Python+for+Data+Analysis">Python for Data Analysis</a>

##Please carefully read the following statement and include it in your email:

“I hereby confirm that this submission is my work. I have cited above the origins of any parts of the submission that were taken from Websites, books, forums, blog posts, github repositories, etc. By including this in my email, I understand that I will be expected to explain my work in a video call with a Udacity coach before I can receive my verified certificate.”

## Is there any other important information that you would want your project evaluator to know?

The code of Lesson 6 has been slightly modified to cope with the version of python (3.x) installed on my computer.

The answer to the programming exercices of Lesson 6 can be found here : [Lesson 6 exercices](Python Solutions to Course Exercises.ipynb)

# 2. <u>Problems Encountered in the Map</u>

The main problem I enconter during this project was a performance issue.  The computer I am using has limited resources and therefore the data extraction and processing task was taking a lot of time.

As a side note, in a near future, I am planning to purshase a <a href="http://bigboards.io/">portable cluster</a> in order to have more processing power.

Next, as we will see later, the data available for Brussels on OpenStreetMap are very poor.

# 3. <u>Data Overview</u>

## 3.1. Load the data in MongoDb

The first step is to generate the JSON file from the OSM file, to do so I run the following command:

In [2]:
from __future__ import absolute_import
from scripts import data
osm_file = "./data/brussels_belgium.osm"

Let's look at the data

````shell
less ./data/brussels_belgium.osm

<?xml version='1.0' encoding='UTF-8'?>
<osm version="0.6" generator="Osmosis 0.43.1">
  <bounds minlon="3.98100" minlat="50.64500" maxlon="4.76100" maxlat="51.05300" origin="http://www.openstreetmap.org/api/0.6"/>
  <node id="123350" version="2" timestamp="2011-09-28T10:53:32Z" uid="8196" user="raskas" changeset="9418430" lat="51.0031441" lon="4.3165272"/>
  <node id="123351" version="2" timestamp="2011-09-28T10:53:32Z" uid="8196" user="raskas" changeset="9418430" lat="51.0131105" lon="4.3167801"/>
  <node id="123352" version="1" timestamp="2007-05-31T20:44:09Z" uid="8196" user="raskas" changeset="75136" lat="51.0155716" lon="4.3174118"/>
  <node id="123353" version="3" timestamp="2011-09-28T10:53:32Z" uid="8196" user="raskas" changeset="9418430" lat="51.017643" lon="4.3184069"/>
  <node id="123354" version="2" timestamp="2011-09-28T10:53:32Z" uid="8196" user="raskas" changeset="9418430" lat="51.0260773" lon="4.3226409"/>
  <node id="123355" version="2" timestamp="2011-10-28T17:40:00Z" uid="138772" user="lodde1949" changeset="9678621" lat="51.0434516" lon="4.3314061"/>
  <node id="125892" version="6" timestamp="2011-11-24T22:05:32Z" uid="15399" user="eMerzh" changeset="9938190" lat="50.8500083" lon="4.3553223"/>
  <node id="125893" version="6" timestamp="2011-11-24T22:05:32Z" uid="15399" user="eMerzh" changeset="9938190" lat="50.8490037" lon="4.3542701"/>
  <node id="125896" version="7" timestamp="2012-09-07T20:52:24Z" uid="383309" user="AtonX" changeset="13027910" lat="50.8434399" lon="4.3445588"/>
  <node id="125898" version="5" timestamp="2010-12-03T08:51:32Z" uid="8323" user="Toi" changeset="6523562" lat="50.8488054" lon="4.3549931"/>
  <node id="125901" version="6" timestamp="2014-10-25T11:09:21Z" uid="15188" user="Polyglot" changeset="26318656" lat="50.850879" lon="4.3394055"/>
  <node id="125904" version="4" timestamp="2014-03-02T09:17:54Z" uid="101145" user="BenoitL" changeset="20860480" lat="50.8442681" lon="4.3369968"/>
  <node id="125907" version="5" timestamp="2011-07-08T21:44:46Z" uid="246723" user="toSc" changeset="8670047" lat="50.8478484" lon="4.3404787"/>
  <node id="125914" version="5" timestamp="2014-12-23T21:53:46Z" uid="101145" user="BenoitL" changeset="27660812" lat="50.8390406" lon="4.3331801"/>
  <node id="125915" version="18" timestamp="2012-01-24T12:15:35Z" uid="379936" user="Paul-Andre Duchesne" changeset="10483858" lat="50.8396221" lon="4.3247428"/>
  <node id="125916" version="14" timestamp="2014-11-23T21:35:32Z" uid="15188" user="Polyglot" changeset="26983153" lat="50.8372075" lon="4.3186965"/>
brussels_belgium.osm
````

In [ ]:
data = data.process_map(osm_file, False)

This commande is creating a JSON file and we can see the size of both the JSON and the OSM file:
````shell
ls -l data
total 4068384
-rw-r-----@ 1 mic0331  staff   999734572 Feb 19 17:08 brussels_belgium.osm
-rw-r--r--  1 mic0331  staff  1083275201 Mar 11 10:38 brussels_belgium.osm.json
````
Next, we will load the Json document onto mongodb

1/ Start MongoDb locally
````shell
$ mongod --dbpath /data/db
````
2/ Next, we import the data

````shell
mongoimport --db map --collection brussels --file ./data/brussels_belgium.osm.json

connected to: 127.0.0.1
2015-03-11T16:27:22.001+0100 		Progress: 9645967/1083275201	0%
2015-03-11T16:27:22.001+0100 			48700	16233/second
2015-03-11T16:27:25.002+0100 		Progress: 21508812/1083275201	1%
2015-03-11T16:27:25.002+0100 			109500	18250/second
2015-03-11T16:27:28.003+0100 		Progress: 30490110/1083275201	2%
2015-03-11T16:27:28.003+0100 			155300	17255/second
2015-03-11T16:27:31.029+0100 		Progress: 41755620/1083275201	3%
2015-03-11T16:27:31.029+0100 			213100	17758/second
2015-03-11T16:27:34.004+0100 		Progress: 52795325/1083275201	4%
2015-03-11T16:27:34.004+0100 			270200	18013/second
2015-03-11T16:27:37.000+0100 		Progress: 64427537/1083275201	5%

....

2015-03-11T16:31:57.001+0100 		Progress: 1037818151/1083275201	95%
2015-03-11T16:31:57.001+0100 			4720000	16978/second
2015-03-11T16:32:00.004+0100 		Progress: 1055783648/1083275201	97%
2015-03-11T16:32:00.004+0100 			4764200	16954/second
2015-03-11T16:32:03.115+0100 		Progress: 1072011728/1083275201	98%
2015-03-11T16:32:03.115+0100 			4811500	16941/second
2015-03-11T16:32:05.435+0100 check 9 4845432
2015-03-11T16:32:05.565+0100 imported 4845432 objects
````

## 3.2. Basic statistics and data quality

Next we can start Mongo and get some insight from the OSM file:

````shell
$ mongo
> use map
switched to db map
````

The initial OSM have a size file is 999.7 MB, the generated JSON file is 1.083 GB and the size of the collection is:

````shell
map> db.brussels.stats()
{
	"ns" : "map.brussels",
	"count" : 4845432,
	"size" : 1446876544,
	"avgObjSize" : 298,
	"storageSize" : 1580150784,
	"numExtents" : 19,
	"nindexes" : 1,
	"lastExtentSize" : 415174656,
	"paddingFactor" : 1,
	"systemFlags" : 1,
	"userFlags" : 1,
	"totalIndexSize" : 157216304,
	"indexSizes" : {
		"_id_" : 157216304
	},
	"ok" : 1
}
````
The size of the collection is 1.45 Gb
This value is also confirmed by the following command:

````shell
map> db.brussels.dataSize()
1446876544
````
Next, we will explore the size of some attributes of the dataset
####Number of documents in the dataset
````shell
map> db.brussels.count()
4845432
````
####Number of contributor to this map
````shell
map> db.brussels.distinct("created.user").length
2061
````
####Number of nodes and ways
````shell
map> db.brussels.find( { type: "node" } ).length()
4142130
map> db.brussels.find( { type: "way" } ).length()
702693
````
####Number of shops
````shell
map> db.brussels.find( { shop: { $exists: true } } ).count()
4378
````

####Number of Café
````shell
map> db.brussels.find( { amenity: "cafe" } ).count()
485
````

Based on these simple query we can say that the data quality is relatively poor.  Knowing a bit Brussels, I know there are lot more café and shops in this region than the number return from these query.  This indicate that the data from OpenStreetMap  are incompleted or not up-to-date.

Let's now look at look at the data a bit deeply, we will check the record containing an address entry:
````shell
map> db.brussels.find({ "address" : { "$exists" : true } }).pretty()
...
{
	"_id" : ObjectId("55005ed88254b358d236c9ef"),
	"created" : {
		"uid" : "126124",
		"version" : "23",
		"user" : "le loup qui rit",
		"changeset" : "22924462",
		"timestamp" : "2014-06-14T12:02:08Z"
	},
	"operator" : "NMBS/SNCB",
	"name:nl" : "Brussel-Noord",
	"name:en" : "Brussels-North",
	"name:fr" : "Bruxelles-Nord",
	"address" : {
		"country" : "BE"
	},
	"pos" : [
		50.8601323,
		4.361512
	],
	"id" : "66180313",
	"type" : "node",
	"name:de" : "Brüssel-Nord",
	"name" : "Bruxelles-Nord - Brussel-Noord",
	"railway" : "station",
	"uic_ref" : "8800002",
	"ref" : "FBN"
}
{
	"_id" : ObjectId("55005ed88254b358d236c9f0"),
	"created" : {
		"uid" : "15399",
		"version" : "6",
		"user" : "eMerzh",
		"changeset" : "19472757",
		"timestamp" : "2013-12-15T21:46:21Z"
	},
	"old_name:fr" : "Haeren",
	"is_in" : "Europe, Belgium, Bruxelles-Capitale, Ville de Bruxelles, Haren",
	"address" : {
		"postcode" : "1130"
	},
	"population:date" : "2005",
	"id" : "66186335",
	"old_name:nl" : "Haaren",
	"place" : "village",
	"type" : "node",
	"name" : "Haren",
	"pos" : [
		50.8909441,
		4.4157553
	],
	"wikipedia" : "fr:Haren (Bruxelles)",
	"population" : "4269",
	"old_name" : "Haeren / Haaren"
}
...
````


Again we can see from these 2 nodes that the data quality is very poor.  The first node is a train station quite well known in Belgium and the fact that the address detail is limited to the country code show clear evidence of lack of information.  However, it is interesting to note that in Belgium we have 3 national languages (French, German, Flamish).  Here we see a translation available for the main language, Flamish, French and English.

Let's now look at the top 5 appearing amenities

````shell
db.brussels.aggregate([
        {"$match":{"amenity":{"$exists":1}}}, 
        {"$group":{"_id":"$amenity", "count":{"$sum":1}}}, 
        {"$sort":{"count":1}}, 
        {"$limit":5}
])

{ "_id" : "music_venue", "count" : 1 }
{ "_id" : "vehicle_inspection", "count" : 1 }
{ "_id" : "scout_hut", "count" : 1 }
{ "_id" : "boat_storage", "count" : 1 }
{ "_id" : "exhibition_center", "count" : 1 }
````

Again, we don't see any multiple occurance of the same `amenity` showing poor data quality.
Finally, we can run another query to see what are the top names in the dataset :

````shell
map> db.brussels.aggregate([
           {"$group":{"_id": "$name", "count": {"$sum": 1}}},
           {"$sort": {"count": -1}},
           {"$limit": 5}
])

{ "_id" : null, "count" : 4754063 }
{ "_id" : "Brusselsesteenweg", "count" : 368 }
{ "_id" : "Mechelsesteenweg", "count" : 240 }
{ "_id" : "Stationsstraat", "count" : 212 }
{ "_id" : "Leuvensesteenweg", "count" : 211 }
````

Let's see the highest number of restaurant we have in the dataset.

```shell
map> db.brussels.aggregate([
     {"$match":{"amenity":{"$exists":1}, "amenity":"restaurant"}},
     {"$group":{"_id": "$name", "count": {"$sum": 1}}},
     {"$sort": {"count": -1}},
     {"$limit": 5}
])

{ "_id" : null, "count" : 73 }
{ "_id" : "Zorba", "count" : 4 }
{ "_id" : "Lunch Garden", "count" : 4 }
{ "_id" : "Mamma Roma", "count" : 4 }
{ "_id" : "De Met", "count" : 3 }
```

# 4. Additional Ideas

Due to the fact that this dataset is really poor, I have created a dedicated section where I can explore the data auditing part of the project.  This sub-project can be found at this link : [NY data anlysis](Data Wrangling on the NY dataset.ipynb)

Why not exploring the data quality arround a certain geographical position point.  We will use the `$geoNear` command of mango to do this exercice.  At Brussel, there is a famous place called `Grand-Place`, the latitude and longitude of this position are respectively 50,84 and 4,35. Here is the exact position from Google Map : <a href="https://www.google.be/maps/place/Grand-Place/@50.846868,4.351788,15z/data=!4m2!3m1!1s0x47c3c47f4614f1f1:0xb03c355d8fe2cfb6?hl=fr">Grand place</a>

Let's build a query:

````shell
# Build a 2D index for the lat/long
map> db.brussels.ensureIndex({pos: "2d"})
{
	"createdCollectionAutomatically" : false,
	"numIndexesBefore" : 1,
	"numIndexesAfter" : 2,
	"ok" : 1
}
# Find the object arround the Grand-Place for a maxDistance of 1km
# By default mongodb $near accepts $maxDistance as radius
# One degree is approximately 111.12 kilometers
map> db.brussels.find({
    pos: {$near:[50.84, 4.35], $maxDistance: 1/111.12}, 
    "address" : { "$exists" : true }
}).pretty()

{
	"_id" : ObjectId("55005fa78254b358d26e78fb"),
	"created" : {
		"uid" : "15399",
		"version" : "1",
		"user" : "eMerzh",
		"changeset" : "20485240",
		"timestamp" : "2014-02-10T13:31:49Z"
	},
	"type" : "node",
	"address" : {
		"street" : "Rue Blaes - Blaesstraat",
		"housenumber" : "40"
	},
	"pos" : [
		50.8399661,
		4.3498853
	],
	"id" : "2665166241",
	"ref:UrbIS" : "7876663"
}
{
	"_id" : ObjectId("55005fa78254b358d26e7925"),
	"created" : {
		"uid" : "15399",
		"version" : "1",
		"user" : "eMerzh",
		"changeset" : "20485240",
		"timestamp" : "2014-02-10T13:31:51Z"
	},
	"type" : "node",
	"address" : {
		"street" : "Rue Blaes - Blaesstraat",
		"housenumber" : "46"
	},
	"pos" : [
		50.8399257,
		4.3498449
	],
	"id" : "2665166292",
	"ref:UrbIS" : "7876663"
}

````
We see that the data are also relatively poor knowing the density of shop, restaurant etc. arround this place.  

Let's look if the query return proper value from the `geoNear` command which allow to obtain the distances between objects in a 2D cartesian space.

<img src="pictures/GrandPlace.png">

We see that the `"street" : "Rue Blaes - Blaesstraat"` is actually very close (1.4 km) to the `Grand-place` so the query does make sence.

# 5. Conclusion

After this review of the data it’s obvious that the Brussels area is incomplete.
Maybe a possible way to improove this dataset is making a script that will extract useful data from the Google Maps API and validate the information available in OpenStreetMap.  Unfortunately, this might violate the Google Maps policy.

Nevertheless, it is nice to see open initiative such as OpenStreeMap, for me personally, it open a lot of possibility just for practicing data cleaning, asses the quality of the data for validity, accuracy, completness, consistency ...

This project gives a solid fundation on the steps needed to wrangle a dataset.  This workflow can be summerized in 7 steps.

    1/ Grab the data of concern (XML, website, JSON ...)
    2/ Parse these data, get an understanding of the data, find documentation to gain domain knowledge
    3/ Explore the data by checking for anomalies
    4/ Audit the data, improove the data quality by building clever data corrector mechanism
    5/ Produce a usable output (here JSON)
    6/ Load the data to a database
    7/ query the data to get insight and start doing data analysis

These are basically the steps applied during any ETL (Extract-Transform-Load) project.

#6. Reference

<ul>
    <li><a href="https://wiki.openstreetmap.org/wiki/OSM_XML">OSM XML Documentation</a></li>
    <li><a href="http://wiki.openstreetmap.org/wiki/Main_Page">OpenStreetMap Documentation</a></li>
    <li><a href="https://mapzen.com/metro-extracts/">Mapzen</a></li>
</ul>